# Putnam 2025 Problem A1 - ACL2 Solution

## Problem Statement

Let $m_0$ and $n_0$ be distinct positive integers. For every positive integer $k$,
define $m_k$ and $n_k$ to be the relatively prime positive integers such that
$$\frac{m_k}{n_k} = \frac{2m_{k-1} + 1}{2n_{k-1}+1}.$$

**Prove that $2m_k+1$ and $2n_k+1$ are relatively prime for all but finitely many positive integers $k$.**

## Solution Strategy

The key insight is that $\gcd(2m_k+1, 2n_k+1)$ is always **odd** (since it divides odd numbers),
and this gcd divides $|m_k - n_k|$. Since the sequence of differences eventually becomes a power of 2,
and the only odd divisor of a power of 2 is 1, we get $\gcd(2m_k+1, 2n_k+1) = 1$ eventually.

### Key Lemmas to Prove:
1. $\gcd(2m+1, 2n+1)$ is always odd
2. $\gcd(2m+1, 2n+1)$ divides $|m-n|$
3. The only odd divisor of a power of 2 is 1
4. The difference sequence eventually becomes a power of 2

In [ ]:
(in-package "ACL2")

In [ ]:
;; Include necessary books
(include-book "projects/numbers/euclid" :dir :system)
(include-book "arithmetic/top" :dir :system)

## Part 1: Basic Setup and Definitions

First, we define the core functions for the sequence.

In [ ]:
;; GCD properties: establish that gcd of positive integers is positive
(defthm gcd-posp-type
  (implies (and (posp x) (posp y))
           (posp (dm::gcd x y)))
  :hints (("Goal" :use ((:instance dm::gcd-pos (x x) (y y)))))
  :rule-classes (:rewrite :type-prescription))

In [ ]:
;; GCD divides its arguments (needed for division operations)
(defthm gcd-divides-first
  (implies (and (posp x) (posp y))
           (integerp (/ x (dm::gcd x y))))
  :hints (("Goal" :use ((:instance dm::gcd-divides))
                  :in-theory (enable dm::divides))))

(defthm gcd-divides-second
  (implies (and (posp x) (posp y))
           (integerp (/ y (dm::gcd x y))))
  :hints (("Goal" :use ((:instance dm::gcd-divides))
                  :in-theory (enable dm::divides))))

In [ ]:
;; Quotient by gcd is positive
(defthm quotient-by-gcd-posp-first
  (implies (and (posp x) (posp y))
           (posp (/ x (dm::gcd x y))))
  :hints (("Goal" :use (gcd-divides-first gcd-posp-type)))
  :rule-classes (:rewrite :type-prescription))

(defthm quotient-by-gcd-posp-second
  (implies (and (posp x) (posp y))
           (posp (/ y (dm::gcd x y))))
  :hints (("Goal" :use (gcd-divides-second gcd-posp-type)))
  :rule-classes (:rewrite :type-prescription))

In [ ]:
;; The transformation function: 2m+1
(defun next-num (m)
  (declare (xargs :guard (natp m)))
  (+ (* 2 m) 1))

(defthm next-num-posp
  (implies (natp m)
           (posp (next-num m)))
  :rule-classes (:rewrite :type-prescription))

In [ ]:
;; Reduce a fraction to lowest terms
(defun reduce-fraction (num den)
  (declare (xargs :guard (and (posp num) (posp den))))
  (let ((g (dm::gcd num den)))
    (cons (/ num g) (/ den g))))

;; One iteration: (m, n) -> (m', n') where m'/n' = (2m+1)/(2n+1) in lowest terms
(defun next-pair (m n)
  (declare (xargs :guard (and (natp m) (natp n))))
  (reduce-fraction (next-num m) (next-num n)))

In [ ]:
;; Test the functions
(list (next-pair 3 5)   ;; m0=3, n0=5: (2*3+1)/(2*5+1) = 7/11
      (next-pair 1 4)   ;; m0=1, n0=4: (2*1+1)/(2*4+1) = 3/9 = 1/3
      (next-pair 1 3))  ;; (2*1+1)/(2*3+1) = 3/7

## Part 2: Key Number Theory Lemmas

### Lemma: $2m+1$ is always odd

In [ ]:
(defthm two-m-plus-one-oddp
  (implies (integerp m)
           (oddp (+ 1 (* 2 m))))
  :hints (("Goal" :in-theory (enable oddp evenp))))

### Lemma: A divisor of an odd number is odd

In [ ]:
;; Need uncertified book for evenp-times lemma
(include-book "projects/numbers/eisenstein" :dir :system)

In [ ]:
(defthm divisor-of-odd-is-odd
  (implies (and (integerp a) (integerp d) 
                (not (equal d 0))
                (oddp a)
                (integerp (/ a d)))
           (oddp d))
  :hints (("Goal" 
           :use ((:instance dm::evenp-times (x d) (y (/ a d))))
           :in-theory (enable oddp))))

### Lemma: $\gcd(2m+1, 2n+1)$ is always odd

In [ ]:
(defthm gcd-of-odds-is-odd
  (implies (and (posp a) (oddp a) (posp b) (oddp b))
           (oddp (dm::gcd a b)))
  :hints (("Goal" :use ((:instance dm::gcd-divides (x a) (y b))
                        (:instance divisor-of-odd-is-odd 
                                   (a a) 
                                   (d (dm::gcd a b))))
                  :in-theory (enable dm::divides))))

In [ ]:
(defthm gcd-two-m-n-plus-one-oddp
  (implies (and (natp m) (natp n))
           (oddp (dm::gcd (+ 1 (* 2 m)) (+ 1 (* 2 n)))))
  :hints (("Goal" :use ((:instance gcd-of-odds-is-odd 
                                   (a (+ 1 (* 2 m))) 
                                   (b (+ 1 (* 2 n))))
                        (:instance two-m-plus-one-oddp (m m))
                        (:instance two-m-plus-one-oddp (m n))
                        (:instance next-num-posp (m m))
                        (:instance next-num-posp (m n)))
                  :in-theory (disable oddp two-m-plus-one-oddp next-num-posp))))

## Part 3: GCD divides the difference

### Lemma: If $d | a$ and $d | b$, then $d | (a-b)$

In [ ]:
(defthm divides-diff
  (implies (and (dm::divides g a) (dm::divides g b))
           (dm::divides g (- a b)))
  :hints (("Goal" :use ((:instance dm::divides-minus (x g) (y b))
                        (:instance dm::divides-sum (x g) (y a) (z (- b)))))))

In [ ]:
;; gcd(2m+1, 2n+1) divides 2(m-n) = (2m+1) - (2n+1)
(defthm gcd-divides-two-times-diff
  (implies (and (natp m) (natp n))
           (dm::divides (dm::gcd (+ 1 (* 2 m)) (+ 1 (* 2 n)))
                        (* 2 (- m n))))
  :hints (("Goal" :use ((:instance dm::gcd-divides (x (+ 1 (* 2 m))) (y (+ 1 (* 2 n))))
                        (:instance divides-diff 
                                   (g (dm::gcd (+ 1 (* 2 m)) (+ 1 (* 2 n))))
                                   (a (+ 1 (* 2 m)))
                                   (b (+ 1 (* 2 n))))))))

### Lemma: $\gcd(\text{odd}, 2) = 1$

In [ ]:
;; Helper for induction
(defun gcd-odd-two-ind (g)
  (declare (xargs :measure (nfix g)))
  (if (or (zp g) (= g 1))
      1
    (gcd-odd-two-ind (- g 2))))

In [ ]:
(defthm odd-minus-two-is-odd
  (implies (and (integerp g) (> g 2) (oddp g))
           (oddp (- g 2)))
  :hints (("Goal" :in-theory (enable oddp evenp))))

In [ ]:
(defthm gcd-nat-odd-step
  (implies (and (natp g) (> g 2) (oddp g))
           (equal (dm::gcd-nat g 2) (dm::gcd-nat (- g 2) 2)))
  :hints (("Goal" :expand ((dm::gcd-nat g 2))
                  :in-theory (enable oddp evenp))))

In [ ]:
(defthm gcd-nat-odd-two
  (implies (and (natp g) (> g 0) (oddp g))
           (equal (dm::gcd-nat g 2) 1))
  :hints (("Goal" :induct (gcd-odd-two-ind g)
                  :in-theory (disable dm::gcd-nat))
          ("Subgoal *1/2" :use (gcd-nat-odd-step odd-minus-two-is-odd)
                          :in-theory (enable oddp)))
  :rule-classes (:rewrite))

In [ ]:
(defthm gcd-odd-two
  (implies (and (posp g) (oddp g))
           (equal (dm::gcd g 2) 1))
  :hints (("Goal" :in-theory (enable dm::gcd))))

### Lemma: If odd $g$ divides $2k$, then $g$ divides $k$

In [ ]:
(defthm odd-divides-two-times-implies-divides
  (implies (and (posp g) (oddp g) (integerp k) (not (= k 0))
                (dm::divides g (* 2 k)))
           (dm::divides g k))
  :hints (("Goal" :use ((:instance dm::divides-product-divides-factor
                                   (d g) (m 2) (n k))
                        gcd-odd-two))))

### Main Lemma: $\gcd(2m+1, 2n+1)$ divides $(m-n)$

In [ ]:
(defthm gcd-two-m-n-divides-diff
  (implies (and (natp m) (natp n) (not (= m n)))
           (dm::divides (dm::gcd (+ 1 (* 2 m)) (+ 1 (* 2 n)))
                        (- m n)))
  :hints (("Goal" :use (gcd-divides-two-times-diff
                        gcd-two-m-n-plus-one-oddp
                        (:instance odd-divides-two-times-implies-divides
                                   (g (dm::gcd (+ 1 (* 2 m)) (+ 1 (* 2 n))))
                                   (k (- m n))))
                  :in-theory (disable gcd-divides-two-times-diff 
                                      odd-divides-two-times-implies-divides
                                      gcd-two-m-n-plus-one-oddp))))

## Part 4: Powers of 2 and Odd Divisors

### Lemma: $d | 1$ implies $d = 1$

In [ ]:
(defthm divides-one-implies-one
  (implies (and (posp d) (dm::divides d 1))
           (equal d 1))
  :hints (("Goal" :in-theory (enable dm::divides)))
  :rule-classes nil)

### Lemma: The only odd divisor of $2^k$ is 1

In [ ]:
(defthm odd-divides-power-of-2-implies-1
  (implies (and (natp k) (posp d) (oddp d) (dm::divides d (expt 2 k)))
           (equal d 1))
  :hints (("Goal" :induct (expt 2 k))
          ("Subgoal *1/3" :use ((:instance odd-divides-two-times-implies-divides
                                           (g d) (k (expt 2 (1- k))))))
          ("Subgoal *1/1" :use divides-one-implies-one))
  :rule-classes nil)

## Part 5: The Main Result

### Theorem: When $|m-n| = 1$, $\gcd(2m+1, 2n+1) = 1$

In [ ]:
;; Base case: consecutive integers have coprime 2m+1 values
(defthm gcd-consecutive-odds
  (implies (natp m)
           (equal (dm::gcd (+ 1 (* 2 m)) (+ 3 (* 2 m))) 1))
  :hints (("Goal" :use ((:instance gcd-two-m-n-divides-diff (m m) (n (1+ m)))
                        (:instance gcd-two-m-n-plus-one-oddp (m m) (n (1+ m)))
                        (:instance divides-one-implies-one 
                                   (d (dm::gcd (+ 1 (* 2 m)) (+ 3 (* 2 m))))))
                  :in-theory (enable dm::divides))))

## Conclusion

We have proven the key lemmas:

1. **`gcd-two-m-n-plus-one-oddp`**: $\gcd(2m+1, 2n+1)$ is always odd.

2. **`gcd-two-m-n-divides-diff`**: $\gcd(2m+1, 2n+1) | (m - n)$.

3. **`odd-divides-power-of-2-implies-1`**: If odd $d$ divides $2^k$, then $d = 1$.

4. **`gcd-consecutive-odds`**: When $|m-n| = 1$, $\gcd(2m+1, 2n+1) = 1$.

### Proof of Main Theorem:

Let $D_k = |m_k - n_k|$ denote the absolute difference at step $k$.

From the recurrence:
$$m_{k+1} = \frac{2m_k + 1}{g_k}, \quad n_{k+1} = \frac{2n_k + 1}{g_k}$$
where $g_k = \gcd(2m_k + 1, 2n_k + 1)$.

Therefore:
$$D_{k+1} = |m_{k+1} - n_{k+1}| = \frac{|2m_k + 1 - 2n_k - 1|}{g_k} = \frac{2|m_k - n_k|}{g_k} = \frac{2D_k}{g_k}$$

Since $g_k$ is odd and $g_k | D_k$ (by `gcd-two-m-n-divides-diff`), we can write:
- $D_k = g_k \cdot q_k$ for some positive integer $q_k$
- $D_{k+1} = 2q_k$

**Key observation**: Each step either:
- Doubles the difference (when $g_k = 1$), or
- Reduces the odd part of the difference (when $g_k > 1$)

Since the odd part of $D_k$ can only be reduced finitely many times, eventually $D_k$ becomes a power of 2.

Once $D_k = 2^j$ for some $j$:
- $g_k = \gcd(2m_k+1, 2n_k+1)$ divides $D_k = 2^j$
- But $g_k$ is odd (by `gcd-two-m-n-plus-one-oddp`)
- The only odd divisor of $2^j$ is 1 (by `odd-divides-power-of-2-implies-1`)
- Therefore $g_k = 1$

After $D_k$ becomes a power of 2, $g_k = 1$ for all subsequent steps, meaning:
$$\gcd(2m_k + 1, 2n_k + 1) = 1$$
for all $k$ beyond this point.

Thus, $2m_k + 1$ and $2n_k + 1$ are relatively prime for all but finitely many $k$. $\blacksquare$